In [2]:
!python inference.py -m samples/5eed8296d6b96_60dac2a791cc1.flac -c configs/train_rnn.yml -w weights/DualPath_RNN_179_-3.1895.pt -s samples

saved in: samples/spk1/inferenced_5eed8296d6b96_60dac2a791cc1.flac
saved in: samples/spk2/inferenced_5eed8296d6b96_60dac2a791cc1.flac
